Задача 1. Разовьем тему с бойцами. Напишите игру (можно взять свой старый код в качестве базы), где игроку будет предложено выбрать класс героя: волшебник или боец. Вы должны учесть возможность добавления новых игровых классов (используйте наследование). У волшебника и бойца немного разные атрибуты (можно атрибуты сделать одинаковые в классе-родителе, но разные коэффициенты в классах-детях, на которые они домножаются: например, здоровье волшебника будет 1.0 от стандартного значения, а здоровье бойца - 1.5, а с маной наоборот). Также у них будут разные методы "нанести удар" и, если хотите, приветствия. Также у нашего героя, кем бы он ни был, должен быть рюкзак, в котором можно рыться и хранить ограниченный набор вещей (в частности, там хранятся зелья: по умолчанию пусть в начале игры каждому персонажу дается по три зелья). Наконец, нужен класс для противника: можете придумать любого монстра (тоже с возможностью добавления новых монстров, очевидно), с которым герой будет сражаться. Во время сражения неплохо предоставлять игроку выбор вида "нанести удар - выпить зелье", а сам урон от удара можно немного рандомизировать с помощью одноименного модуля. Можно еще реализовать и метод sleep для мирного времени, но во время боя он, конечно, не понадобится.

In [253]:
import random

In [302]:
class Hero:
    def __init__(self, name, health_coef, mana_coef):
        self.name = name
        self.base_health = 70
        self.base_mana = 70
        self.health = self.base_health * health_coef
        self.mana = self.base_mana * mana_coef
        self.backpack = Backpack()
    def attack(self):
        raise NotImplementedError
    def loss(self,damage):
        self.health -= damage
        if self.health < 0:
            self.health = 0 # чтобы не значение не уходило в минус
        print(f'Теперь уровень здоровья {self.name} составляет {self.health}')
    def __str__(self):
        return f'{self.name} (Здоровье: {self.health}, Мана: {self.mana})'
    

class Wizard(Hero):
    def __init__(self, name):
        health_coef = 1
        mana_coef = 1.5
        super().__init__(name, health_coef, mana_coef)
    def attack(self, enemy):
        damage = random.randint(10, 25)
        print(f'Своим заклинанием {self.name} наносит урон {damage}')
        self.mana -= 0.5 * damage
        enemy.loss(damage)
        if damage >= 22: # За особо сильный удар в награду дается одно зелье
            self.backpack['зелья'] += 1
            print('Вы получили в награду одно зелье')

class Warrior(Hero):
    def __init__(self, name):
        health_coef = 1.5
        mana_coef = 1
        super().__init__(name, health_coef, mana_coef)
    def attack(self, enemy):
        damage = random.randint(13, 28)
        print(f'Своим мечом {self.name} наносит урон {damage}')
        if damage >= 25:
            self.mana -= 2
        enemy.loss(damage)
        if damage >= 25:
            self.backpack['зелья'] += 1
            print('Вы получили в награду одно зелье')
            

In [303]:
class Enemy:
    def __init__(self, health):
        self.health = health
    def loss(self, damage):
        self.health -= damage
        if self.health < 0:
            self.health = 0
        print(f'Теперь уровень здоровья врага составляет {self.health}')
    def attack(self):
        raise NotImplementedError
    
class Goblin(Enemy):
    def __init__(self):
        health = 80
        super().__init__(health)
    def attack(self, hero):
        damage = random.randint(5, 25)
        print(f'Гоблин наносит урон {damage}')
        hero.loss(damage)
    

In [304]:
class Backpack:
    def __init__(self):
        self.items = {'зелья': 0}

    def use_potion(self):
        if self.items['зелья'] > 0:
            self.items['зелья'] -= 1
            hupgrade = 20 # добавляется к здоровью при использовании зелья
            mupgrade = 10 # добывляется к мане
            print(f'Использовано зелье: здоровье + {hupgrade}, мана + {mupgrade}')
            return hupgrade, mupgrade
        else:
            print('Зелья закончились')
            hupgrade = 0
            mupgrade = 0

    def __getitem__(self, item):
        return self.items.get(item, 0)

    def __setitem__(self, item, count):
        self.items[item] = count

    def __iter__(self):
        return iter(self.items.items())
    

In [305]:
class Game:
    def __init__(self):
        self.hero = None
        
        
        
    def play(self):
        
        enemy = Goblin()
        hero_choice = input('Выберите персонажа:1. Волшебник 2. Боец')
        hero_name = input('Введите имя')
        if hero_choice == '1':
          self.hero = Wizard(hero_name)
        elif hero_choice == '2':
          self.hero = Warrior(hero_name)
        else:
           print('Некорректный выбор')
        self.hero.backpack['зелья'] = 3  
        potions_available = self.hero.backpack['зелья']
        print(f'У вас есть {potions_available} зелий')  
        print('Игра началась')
        while enemy.health > 0 and self.hero.health > 0:
           choice = input('Выберите действие: 1.Нанести удар 2. Выпить зелье')
           if choice == '1':
              damage = self.hero.attack(enemy)
           elif choice == '2':
              hupgrade, mupgrade = self.hero.backpack.use_potion()
              self.hero.health += hupgrade
              self.hero.mana += mupgrade
              print(self.hero)
           else:
                print('Неверный выбор действия')
                continue
           for item, count in self.hero.backpack:
              print(f'{item}: {count}')
           print(self.hero)
           

           if enemy.health > 0:
                enemy.attack(self.hero)

        if self.hero.health <= 0:
            print(f'{self.hero.name} погиб!')
        else:
            print(f'{self.hero.name} победил врага!')

              
              
              
        

In [306]:
game = Game()
game.play()

У вас есть 3 зелий
Игра началась
Своим заклинанием Bob наносит урон 18
Теперь уровень здоровья врага составляет 62
зелья: 3
Bob (Здоровье: 70, Мана: 96.0)
Гоблин наносит урон 20
Теперь уровень здоровья Bob составляет 50
Своим заклинанием Bob наносит урон 18
Теперь уровень здоровья врага составляет 44
зелья: 3
Bob (Здоровье: 50, Мана: 87.0)
Гоблин наносит урон 23
Теперь уровень здоровья Bob составляет 27
Использовано зелье: здоровье + 20, мана + 10
Bob (Здоровье: 47, Мана: 97.0)
зелья: 2
Bob (Здоровье: 47, Мана: 97.0)
Гоблин наносит урон 13
Теперь уровень здоровья Bob составляет 34
Своим заклинанием Bob наносит урон 13
Теперь уровень здоровья врага составляет 31
зелья: 2
Bob (Здоровье: 34, Мана: 90.5)
Гоблин наносит урон 5
Теперь уровень здоровья Bob составляет 29
Своим заклинанием Bob наносит урон 22
Теперь уровень здоровья врага составляет 9
Вы получили в награду одно зелье
зелья: 3
Bob (Здоровье: 29, Мана: 79.5)
Гоблин наносит урон 21
Теперь уровень здоровья Bob составляет 8
Использо

Зададча 2. Вспомним задачу токенизации. Напишите собственный простенький токенизатор (с самим процессом можно не сильно заморачиваться), который будет создавать генератор с объектами класса Token, у которых будет атрибут text и атрибут category (латинское слово, кириллическое слово или пунктуация). Токенизатор должен быть реализован в классе, у которого должна быть (генераторная) функция tokenize(). Вам понадобится отдельный класс для токенов и re.finditer().

In [299]:
import re

class Token:
    def __init__(self, text, category):
        self.text = text
        self.category = category
    def __repr__(self):
        return f"'{self.text}' : {self.category}"

class Tokenizer:
    def __init__(self):
        self.pattern = re.compile(r'([a-zA-Z]+|[а-яА-ЯёЁ]+|[.,!?;:()"-])')
    def tokenize(self, text):
        matches = re.finditer(self.pattern, text)
        for match in matches:
            token = match.group(0) 
            if re.match(r'[a-zA-Z]+', token):
                category = 'латинское слово'
            elif re.match(r'[а-яА-ЯёЁ]+', token):
                category = 'кириллическое слово'
            else:
                category = 'пунктуация'

            yield Token(token, category)

            

In [301]:
tokenizer = Tokenizer()
text = input()
tokens = tokenizer.tokenize(text)

for token in tokens:
    print(token)


'Катя' : кириллическое слово
',' : пунктуация
'Лена' : кириллическое слово
'и' : кириллическое слово
'Женя' : кириллическое слово
'пошли' : кириллическое слово
'в' : кириллическое слово
'магазин' : кириллическое слово
'Gloria' : латинское слово
'Jeans' : латинское слово
'!' : пунктуация


Вы разрабатываете игру, в которой главный герой — археолог и исследователь древних цивилизаций. Его цель — собрать и исследовать артефакты, которые разбросаны по разным древним руинам. Ваша задача — создать класс ArtifactCollector, который будет управлять коллекцией артефактов. Эта коллекция должна быть итерируемой, поддерживать сортировку и фильтрацию артефактов по разным критериям.

In [220]:
class Artifact:
    def __init__(self, name, age, origin, rarity):
        self.name = name
        self.age = age
        self.origin = origin
        self.rarity = rarity


class ArtifactCollector:
    def __init__(self):
        self.artifacts = []
    def add_artifact(self, artifact):
        self.artifacts.append(artifact)
    def __iter__(self):
        return iter(self.artifacts)
    def sort_by_age(self):
        self.artifacts = sorted(self.artifacts, key=lambda artifact: artifact.age)
    def filter_by_origin(self, origin):
        return [artifact for artifact in self.artifacts if artifact.origin == origin]
    def filter_by_rarity(self, rarity):
        return [artifact for artifact in self.artifacts if artifact.rarity == rarity]
    def __reversed__(self):
        return iter(reversed(self.artifacts))
    def find_oldest(self):
        return max(self.artifacts, key=lambda artifact: artifact.age)
    
    def find_by_name(self, name):
        return next((artifact for artifact in self.artifacts if artifact.name == name), None)




In [221]:
mask = Artifact("Golden Mask", 3000, "Egypt", "legendary")
vase = Artifact("Ancient Vase", 2000, "Greece", "rare")
statue = Artifact("Stone Statue", 1500, "Aztec", "common")
amulet = Artifact("Amulet of Anubis", 5000, "Egypt", "legendary")

In [222]:
collector = ArtifactCollector()
collector.add_artifact(mask)
collector.add_artifact(vase)
collector.add_artifact(statue)
collector.add_artifact(amulet)

In [223]:
print("Все артефакты:")
for artifact in collector:
    print(artifact.name)

Все артефакты:
Golden Mask
Ancient Vase
Stone Statue
Amulet of Anubis


In [224]:
print("\nАртефакты в обратном порядке:")
for artifact in reversed(collector):
    print(artifact.name)


Артефакты в обратном порядке:
Amulet of Anubis
Stone Statue
Ancient Vase
Golden Mask


In [225]:
print("\nАртефакты по возрасту:")
collector.sort_by_age()
for artifact in collector:
    print(f"{artifact.name}: {artifact.age} лет")


Артефакты по возрасту:
Stone Statue: 1500 лет
Ancient Vase: 2000 лет
Golden Mask: 3000 лет
Amulet of Anubis: 5000 лет


In [226]:
print("\nАртефакты из Египта:")
for artifact in collector.filter_by_origin("Egypt"):
    print(artifact.name)


Артефакты из Египта:
Golden Mask
Amulet of Anubis


In [227]:
oldest = collector.find_oldest()
print(f"\nСамый древний артефакт: {oldest.name} ({oldest.age} лет)")


Самый древний артефакт: Amulet of Anubis (5000 лет)


In [228]:
artifact = collector.find_by_name("Ancient Vase")
if artifact:
    print(f"\nАртефакт найден: {artifact.name}, {artifact.age} лет, {artifact.origin}")
else:
    print("\nАртефакт не найден")


Артефакт найден: Ancient Vase, 2000 лет, Greece


In [250]:
class Game:
    def __init__(self):
        self.collector = ArtifactCollector()
        self.possible_artifacts = [Artifact("Golden Mask", 3000, "Egypt", "legendary"), Artifact("Ancient Vase", 2000, "Greece", "rare"), Artifact("Stone Statue", 1500, "Aztec", "common"), Artifact("Amulet of Anubis", 5000, "Egypt", "legendary")]
    def choose_artifact(self):
        rarity_choices = {'common': 0.5, 'rare': 0.1, 'legendary': 0.03} # вероятности для артефактов разной степени редкости
        random_value = random.random()
        if random_value < rarity_choices['legendary']:
            rarity = 'legendary'
        elif random_value <  rarity_choices['rare']:
            rarity = 'rare'
        elif random_value <  rarity_choices['common']:
            rarity = 'common'
        else:
            return None
        filtered_artifacts = [artifact for artifact in self.possible_artifacts if artifact.rarity == rarity] # смотрим все что подходит
        if filtered_artifacts:
            chosen_artifact = random.choice(filtered_artifacts) # из этого рандомно выбирается один
            self.collector.add_artifact(chosen_artifact)
            return chosen_artifact

        return None
    def play(self):
        while True:
            choice = input('Выберите действие:\n1. Искать артефакт\n2. Посмотреть все собранные артефакты\n3. Посмотреть последние собранные артефакты\n4. Сортировать артефакты по возрасту\n5. Найти самый древний артефакт\n6. Искать артефакт по имени\n7. Искать артефакты по происхождению\n8. Выйти')
            if choice == '1':
                artifact = self.choose_artifact()
                if artifact:
                    print(f'Найденный артефакт: {artifact.name}, Age: {artifact.age}, Origin: {artifact.origin}, Rarity: {artifact.rarity}')
                else:
                    print('Пока Вы ничего не нашли') # артефакт не обязательно удается найти сразу
            elif choice == '2':
                print('Все артефакты:')
                for artifact in self.collector:
                    print(artifact.name)
            elif choice == '3':
                print('\nАртефакты в обратном порядке:')
                for artifact in reversed(self.collector):
                    print(artifact.name)
            elif choice == '4':
                self.collector.sort_by_age()
                for artifact in self.collector:
                    print(f'Артефакты по возрасту: {artifact.name}: {artifact.age} лет')
            elif choice == '5':
                oldest = self.collector.find_oldest()
                print(f'\nСамый древний артефакт: {oldest.name} ({oldest.age} лет)')
            elif choice == '6':
                name = input('Введите название артефакта:')
                artifact = self.collector.find_by_name(name)
                if artifact:
                    print(f'Артефакт найден: {artifact.name}, Age: {artifact.age}, Origin: {artifact.origin}, Rarity: {artifact.rarity}')
                else:
                    print('Артефакт не найден')
            elif choice == '7':
                origin = input('Введите происхождение: ')
                artifact = self.collector.filter_by_origin(origin)
                if artifact:
                    print(f'Артефакт найден: {artifact.name}, Age: {artifact.age}, Origin: {artifact.origin}, Rarity: {artifact.rarity}')
                else:
                    print('Артефакт не найден')
                
            elif choice == '8':
                break


In [251]:
game = Game()
game.play()

Найденный артефакт: Stone Statue, Age: 1500, Origin: Aztec, Rarity: common
Пока Вы ничего не нашли
Пока Вы ничего не нашли
Пока Вы ничего не нашли
Найденный артефакт: Ancient Vase, Age: 2000, Origin: Greece, Rarity: rare
Пока Вы ничего не нашли
Найденный артефакт: Stone Statue, Age: 1500, Origin: Aztec, Rarity: common
Найденный артефакт: Stone Statue, Age: 1500, Origin: Aztec, Rarity: common
Артефакт не найден
Пока Вы ничего не нашли
Все артефакты:
Stone Statue
Ancient Vase
Stone Statue
Stone Statue
Пока Вы ничего не нашли

Артефакты в обратном порядке:
Stone Statue
Stone Statue
Ancient Vase
Stone Statue
Найденный артефакт: Ancient Vase, Age: 2000, Origin: Greece, Rarity: rare
Найденный артефакт: Stone Statue, Age: 1500, Origin: Aztec, Rarity: common
Артефакты по возрасту: Stone Statue: 1500 лет
Артефакты по возрасту: Stone Statue: 1500 лет
Артефакты по возрасту: Stone Statue: 1500 лет
Артефакты по возрасту: Stone Statue: 1500 лет
Артефакты по возрасту: Ancient Vase: 2000 лет
Артефакты